## Limpieza de Datos: 
## Encuestas de Patrones Infonavit

Carga de librerías:

In [65]:
import os
os.chdir("..")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder,KBinsDiscretizer
from sklearn.impute import SimpleImputer
import sys, os
import re

In [2]:
os.getcwd()

'/home/mariana/Documents/TESIS_MS/DeepLearning_Patrones_Infonavit'

In [3]:
%reload_ext autoreload
%autoreload 
from utils.funcs_utils import *

Lectura de datos:

In [4]:
df= pd.read_csv('/home/mariana/Documents/TESIS_MS/DeepLearning_Patrones_Infonavit/data/BaseAgregada.csv')
df.rename(columns={col: col.lower() for col in df.columns.values},inplace=True)

/home/mariana/.pyenv/versions/deep_learning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (5,21,23,24,26,27,35,37,67,68,69,70,71,74,75,76,77,78,79,80,83,84,85,86,87,88,89,90,92,93,95,96,97,99,100,101,102,105,106,107,108,109,110,124,132,133,143,144,145,146,147,148,149,151,152,153,167,168,169,170,174,176,178,189,190,192,193,195,196,205,210,211) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Se cambia el nombre de las columnas a minúscula:

In [5]:
count_unique_obs(df)

history_id_crm       7834
estrato_nuevo           3
estrato_nuevo_txt       3
ponderador_sgpff       54
pond_relativizado      51
                     ... 
pfin                    1
levantamiento           4
estrato                 7
estrato_txt             7
rfclimpio            7376
Length: 221, dtype: int64

Contamos el total de "sí=1" para ver en cuánto recurso invirtió

In [6]:
is_1= lambda x:int(x==1)

In [7]:
count_19 = lambda row: is_1(row['p2_19_a']) + is_1(row['p2_19_b'])+is_1(row['p2_19_c']) + is_1(row['p2_19_d'])+ is_1(row['p2_19_e']) + is_1(row['p2_19_f'])+ is_1(row['p2_19_g']) + is_1(row['p2_19_h'])+ is_1(row['p2_19_i']) + is_1(row['p2_19_j'])+is_1(row['p2_19_k']) + is_1(row['p2_19_l'])+ is_1(row['p2_19_m']) + is_1(row['p2_19_n'])+is_1(row['p2_19_o']) 
df['p2_19'] = df.apply(count_19 ,axis=1)

In [8]:
count_20 = lambda row: is_1(row['p2_20_a']) + is_1(row['p2_20_b'])+is_1(row['p2_20_c']) + is_1(row['p2_20_d']) 
df['p2_20'] = df.apply(count_20 ,axis=1)

In [9]:
count_19_20 = lambda row: is_1(row['p2_19_a']) + is_1(row['p2_19_b'])+is_1(row['p2_19_c']) + is_1(row['p2_19_d'])+ is_1(row['p2_19_e']) + is_1(row['p2_19_f'])+ is_1(row['p2_19_g']) + is_1(row['p2_19_h'])+ is_1(row['p2_19_i']) + is_1(row['p2_19_j'])+is_1(row['p2_19_k']) + is_1(row['p2_19_l'])+ is_1(row['p2_19_m']) + is_1(row['p2_19_n'])+is_1(row['p2_19_n']) + is_1(row['p2_20_a']) + is_1(row['p2_20_b'])+is_1(row['p2_20_c']) + is_1(row['p2_20_d'])
df['inversion_total'] = df.apply(count_19_20 ,axis=1)

Se suman únicamente los créditos autorizados.

In [10]:
df['2_22_autorizados'] = df['p2_22_a'] + df['p2_22_b'] 

Solicitud de créditos en el trimestre anterior.

In [11]:
df["p2_27_a"] = pd.to_numeric(df.p2_27_a, errors='coerce')
df["p2_27_b"] = pd.to_numeric(df.p2_27_b, errors='coerce')
df["p2_27_c"] = pd.to_numeric(df.p2_27_c, errors='coerce')

In [12]:
count_27 = lambda row: is_1(row['p2_27_a']) + is_1(row['p2_27_b'])+is_1(row['p2_27_c'])
df['p2_27_bis'] = df.apply(count_27,axis=1)

La variables 2_27 establece si en algún mes del trimestre anterior solicitó un crédito es 1, 0 en otro caso.

In [13]:
df['2_27'] = np.where(df['p2_27_bis']>0,1,0)

Si pagó impuestos y/o contribuciones el trimestre anterior: 1=Sí, 2=No.


In [14]:
is_2= lambda x:int(x==2)
is_98= lambda x:int(x==98)
is_99= lambda x:int(x==99)

In [15]:
count_imp = lambda row: is_1(row['p2_41_a']) + is_1(row['p2_41_b'])+is_1(row['p2_41_c'])
df['p2_41_imp'] = df.apply(count_imp ,axis=1)

In [16]:
count_contr= lambda row: is_1(row['p2_41_d']) + is_1(row['p2_41_e'])+is_1(row['p2_41_f'])
df['p2_41_contr'] = df.apply(count_contr ,axis=1)

In [17]:
count_none= lambda row: is_2(row['p2_41_a']) + is_2(row['p2_41_b'])+is_2(row['p2_41_c'])+ is_2(row['p2_41_d']) + is_2(row['p2_41_e'])+is_2(row['p2_41_f'])+ is_2(row['p2_41_g'])
df['p2_41_none'] = df.apply(count_none ,axis=1)

In [18]:
count_imp2 = lambda row: is_2(row['p2_41_a']) + is_2(row['p2_41_b'])+is_2(row['p2_41_c'])
df['p2_41_imp2'] = df.apply(count_imp2 ,axis=1)

In [19]:
count_contr2= lambda row: is_2(row['p2_41_d']) + is_2(row['p2_41_e'])+is_2(row['p2_41_f'])
df['p2_41_contr2'] = df.apply(count_contr2,axis=1)

In [20]:
count_98c= lambda row: is_98(row['p2_41_d']) + is_98(row['p2_41_e'])+is_98(row['p2_41_f'])
df['p2_41_contr98'] = df.apply(count_98c,axis=1)

In [21]:
count_98i= lambda row: is_98(row['p2_41_a']) + is_98(row['p2_41_b'])+is_98(row['p2_41_c'])
df['p2_41_imp98'] = df.apply(count_98i,axis=1)

In [22]:
count_99c= lambda row: is_99(row['p2_41_d']) + is_99(row['p2_41_e'])+is_99(row['p2_41_f'])
df['p2_41_contr99'] = df.apply(count_99c,axis=1)

In [23]:
count_99i= lambda row: is_99(row['p2_41_a']) + is_99(row['p2_41_b'])+is_99(row['p2_41_c'])
df['p2_41_imp99'] = df.apply(count_99i,axis=1)

In [24]:
count_both= lambda row: is_1(row['p2_41_a']) + is_1(row['p2_41_b'])+is_1(row['p2_41_c'])+ is_1(row['p2_41_d']) + is_1(row['p2_41_e'])+is_1(row['p2_41_f'])+ is_1(row['p2_41_g'])
df['p2_41_both'] = df.apply(count_both ,axis=1)

In [25]:
count_g= lambda row: is_1(row['p2_41_g'])
df['2_41_g'] = df.apply(count_g,axis=1)

Si pagó a tiempo los impuestos o contribuciones el trimetre anterior: 1=Sí, 2=No.

In [26]:
count_imp42 = lambda row: is_2(row['p2_42_a']) + is_2(row['p2_42_b'])+is_2(row['p2_42_c'])
df['p2_42_imp'] = df.apply(count_imp42 ,axis=1)

In [27]:
count_contr42= lambda row: is_2(row['p2_42_d']) + is_2(row['p2_42_e'])+is_2(row['p2_42_f'])
df['p2_42_contr'] = df.apply(count_contr42 ,axis=1)

In [28]:
count_all= lambda row: is_2(row['p2_42_a']) + is_2(row['p2_42_b'])+is_2(row['p2_42_c'])+ is_2(row['p2_42_d']) + is_2(row['p2_42_e'])+is_2(row['p2_42_f'])+ is_2(row['p2_42_g'])
df['p2_42_all'] = df.apply(count_all ,axis=1)

In [29]:
count_none42= lambda row: is_1(row['p2_42_a']) + is_1(row['p2_42_b'])+is_1(row['p2_42_c'])+ is_1(row['p2_42_d']) + is_1(row['p2_42_e'])+is_1(row['p2_42_f'])+ is_1(row['p2_42_g'])
df['p2_42_none'] = df.apply(count_none42 ,axis=1)

In [30]:
count_98c= lambda row: is_98(row['p2_42_d']) + is_98(row['p2_42_e'])+is_98(row['p2_42_f'])
df['p2_42_contr98'] = df.apply(count_98c,axis=1)

In [31]:
count_98i= lambda row: is_98(row['p2_42_a']) + is_98(row['p2_42_b'])+is_98(row['p2_42_c'])
df['p2_42_imp98'] = df.apply(count_98i,axis=1)

In [32]:
count_99c= lambda row: is_99(row['p2_42_d']) + is_99(row['p2_42_e'])+is_99(row['p2_42_f'])
df['p2_42_contr99'] = df.apply(count_99c,axis=1)

In [33]:
count_99i= lambda row: is_99(row['p2_42_a']) + is_99(row['p2_42_b'])+is_99(row['p2_42_c'])
df['p2_42_imp99'] = df.apply(count_99i,axis=1)

In [34]:
count_g= lambda row: is_2(row['p2_42_g'])
df['2_42_g2'] = df.apply(count_g,axis=1)

Dificultades para pagar durante este trimestre impuestos y/o contribuciones: 1=Sí, 2=No

In [35]:
count_imp = lambda row: is_1(row['p2_44_a']) + is_1(row['p2_44_b'])+is_1(row['p2_44_c'])
df['p2_44_imp'] = df.apply(count_imp ,axis=1)

In [36]:
count_contr= lambda row: is_1(row['p2_44_d']) + is_1(row['p2_44_e'])+is_1(row['p2_44_f'])
df['p2_44_contr'] = df.apply(count_contr ,axis=1)

In [37]:
count_none= lambda row: is_2(row['p2_44_a']) + is_2(row['p2_44_b'])+is_2(row['p2_44_c'])+ is_2(row['p2_44_d']) + is_2(row['p2_44_e'])+is_2(row['p2_44_f'])
df['p2_44_none'] = df.apply(count_none ,axis=1)

In [38]:
count_both= lambda row: is_1(row['p2_44_a']) + is_1(row['p2_44_b'])+is_1(row['p2_44_c'])+ is_1(row['p2_44_d']) + is_1(row['p2_44_e'])+is_1(row['p2_44_f'])
df['p2_44_both'] = df.apply(count_both ,axis=1)

In [39]:
count_98c= lambda row: is_98(row['p2_44_d']) + is_98(row['p2_44_e'])+is_98(row['p2_44_f'])
df['p2_44_contr98'] = df.apply(count_98c,axis=1)

In [40]:
count_98i= lambda row: is_98(row['p2_44_a']) + is_98(row['p2_44_b'])+is_98(row['p2_44_c'])
df['p2_44_imp98'] = df.apply(count_98i,axis=1)

In [41]:
count_99i= lambda row: is_99(row['p2_44_a']) + is_99(row['p2_44_b'])+is_99(row['p2_44_c'])
df['p2_44_imp99'] = df.apply(count_99i,axis=1)

In [42]:
count_99c= lambda row: is_99(row['p2_44_d']) + is_99(row['p2_44_e'])+is_99(row['p2_44_f'])
df['p2_44_contr99'] = df.apply(count_99c,axis=1)

Dificultades para pagar impuestos y/o contribuciones lo que resta del año: 1=Sí, 2=No

In [43]:
count_imp = lambda row: is_1(row['p2_45_a']) + is_1(row['p2_45_b'])+is_1(row['p2_45_c'])
df['p2_45_imp'] = df.apply(count_imp ,axis=1)

In [44]:
count_contr= lambda row: is_1(row['p2_45_d']) + is_1(row['p2_45_e'])+is_1(row['p2_45_f'])
df['p2_45_contr'] = df.apply(count_contr ,axis=1)

In [45]:
count_none= lambda row: is_2(row['p2_45_a']) + is_2(row['p2_45_b'])+is_2(row['p2_45_c'])+ is_2(row['p2_45_d']) + is_2(row['p2_45_e'])+is_2(row['p2_45_f'])
df['p2_45_none'] = df.apply(count_none ,axis=1)

In [46]:
count_both= lambda row: is_1(row['p2_45_a']) + is_1(row['p2_45_b'])+is_1(row['p2_45_c'])+ is_1(row['p2_45_d']) + is_1(row['p2_45_e'])+is_1(row['p2_45_f'])
df['p2_45_both'] = df.apply(count_both ,axis=1)

In [47]:
count_98c= lambda row: is_98(row['p2_45_d']) + is_98(row['p2_45_e'])+is_98(row['p2_45_f'])
df['p2_45_contr98'] = df.apply(count_98c,axis=1)

In [48]:
count_99c= lambda row: is_99(row['p2_45_d']) + is_99(row['p2_45_e'])+is_99(row['p2_45_f'])
df['p2_45_contr99'] = df.apply(count_99c,axis=1)

In [49]:
count_98i= lambda row: is_98(row['p2_45_a']) + is_98(row['p2_45_b'])+is_98(row['p2_45_c'])
df['p2_45_imp98'] = df.apply(count_98i,axis=1)

In [50]:
count_99i= lambda row: is_99(row['p2_45_a']) + is_99(row['p2_45_b'])+is_99(row['p2_45_c'])
df['p2_45_imp99'] = df.apply(count_99i,axis=1)

In [51]:
a=

SyntaxError: invalid syntax (<ipython-input-51-2d920c342ece>, line 1)

Convertimos todas las variables catagóricas que están como float u object a numéricas para homologar categorías.

In [52]:
pd.set_option('display.max_rows', 10)
convert_to_num(df,vars_to_cat)

,history_id_crm,estrato_nuevo,estrato_nuevo_txt,ponderador_sgpff,pond_relativizado,region,region_txt,empresas,p1_1_a,p1_1_b,p1_2,p1_2_6_tx,p1_2_6_cod,p1_3_tx,p1_3_cod,p1_4,p1_4_9_tx,p1_4_9_cod,p1_5,p2_1_tx1,p2_1_cod11,p2_1_cod12,p2_1_tx2,p2_1_1cod21,p2_1_1cod22,p2_1_tx3,p2_1_2cod31,p2_1_2cod32,p2_2,p2_3,p2_4,p2_5_a,p2_5_b,p2_6,p2_7,p2_8,p2_9,p2_10,p2_11,p2_12,p2_13,p2_14,p2_15,p2_16,p2_17,p2_18,p2_19_a,p2_19_b,p2_19_c,p2_19_d,...,p6_2,p6_3,p6_4_tx,pfin,levantamiento,estrato,estrato_txt,rfclimpio,p2_19,p2_20,inversion_total,2_22_autorizados,p2_27_bis,2_27,p2_41_imp,p2_41_contr,p2_41_none,p2_41_imp2,p2_41_contr2,p2_41_contr98,p2_41_imp98,p2_41_contr99,p2_41_imp99,p2_41_both,2_41_g,p2_42_imp,p2_42_contr,p2_42_all,p2_42_none,p2_42_contr98,p2_42_imp98,p2_42_contr99,p2_42_imp99,2_42_g2,p2_44_imp,p2_44_contr,p2_44_none,p2_44_both,p2_44_contr98,p2_44_imp98,p2_44_imp99,p2_44_contr99,p2_45_imp,p2_45_contr,p2_45_none,p2_45_both,p2_45_contr98,p2_45_contr99,p2_45_imp98,p2_45_imp99
0,7521766,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,,NaN,2,3,2014,2,NaN,2.0,CONSTRUCCIÓN.,3,4,NaN,4.0,2,FALTA DE TRABAJO,6,,PANDEMIA,1,,NC,,,3,98,0,0,0,3,1,1,2,0,2,2,2,1,1,5,2,1,2,2,2,2,...,52,11,NS.,1.0,1Q2021,NaN,NaN,AAA140320F93,0,0,0,0,0,0,2,2,2,1,1,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,3,2,0,5,1,0,0,0,3,3,0,6,0,0,0,0
1,7544090,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,,NaN,2,4,2016,4,NaN,4.0,VENTA DE ELEVADORES E INSTALACIONES,2,3,NaN,3.0,1,ECONOMIA,5,,PANDEMIA,1,,NS,,,10,40,2,9998,0,9998,2,0,2,0,3,2,1,1,2,0,1,1,2,2,2,2,...,32,10,CESAR ULISES PEREZ FLORES,1.0,1Q2021,NaN,NaN,AAR150119HV8,1,0,1,0,0,0,3,3,1,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,6,0,0,0,0,0
2,7543953,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,,NaN,2,98,2006,3,NaN,3.0,FRABRICANTES,2,1,NaN,1.0,2,COVID,1,,FALTA DE LIQUIDES,5,,DESABASTO DE ENTREGA,10,,14,48,0,0,0,14,2,0,2,0,3,2,3,3,1,3,3,3,2,2,2,2,...,48,9,CECILIA ARAIZA,1.0,1Q2021,NaN,NaN,AASC620108FI2,0,0,0,0,0,0,3,3,1,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,6,0,0,0,0,0
3,7536178,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,,NaN,2,98,2007,5,NaN,5.0,SERVICIOS,3,3,NaN,3.0,2,LA ECONOMIA DEL PAIS,7,,LOS CREDITOS QUE NO NOS DAN,5,,LA PANDEMIA,1,,50,40,0,0,4,50,2,0,2,0,3,2,1,1,2,0,1,1,2,2,2,2,...,30,11,ADMINISTRACION,1.0,1Q2021,NaN,NaN,ABA170622I65,0,0,0,0,0,0,3,3,1,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,6,0,0,0,0,0
4,7521922,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,,NaN,2,10,9998,5,NaN,5.0,Limpieza Integral,1,2,NaN,2.0,1,Economia,7,,Pandemia,1,,Inseguridad,9,,80,48,80,0,0,80,1,2,2,0,2,2,2,2,2,0,2,3,2,2,2,2,...,36,11,Hector Gomez Pezuela,1.0,1Q2021,NaN,NaN,ABE150923MA5,3,2,5,20,2,1,3,3,1,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,2,0,4,2,0,0,0,0,2,0,4,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7829,224423829,3,NaN,NaN,0.13,1,Centro Sur,1,7,2017,5,NaN,5.0,servicios de educacion,SERVICIOS SOCIALES Y COMUNALES,8,NaN,8.0,2,98,98,NaN,98,98.0,NaN,98,98.0,NaN,99998,6,2,0,0,58,2,0,2,0,3,2,3,2,2,0,3,3,2,2,2,2,...,25,10,marina,NaN,3Q2021,NaN,NaN,AEB1403149M7,1,0,1,0,0,0,2,2,3,1,1,0,0,0,0,4,0,0,0,0,4,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,6,0,0,0,0,0
7830,224637558,1,NaN,NaN,4.56,1,Centro Sur,1,5,2009,5,NaN,5.0,OUTSOURCING DEPORTIVO,INDUSTRIAS DE TRANSFORMACIÓN,4,NaN,4.0,2,VENTA,2,NaN,PERSONAL,14.0,NaN,PANDEMIA,1.0,NaN,180,5,0,0,0,180,2,0,1,1,3,2,2,1,2,0,1,1,2,2,2,2,...,34,11,JULIO CXESAR QUIROZ,NaN,3Q2021,NaN,NaN,CIQM550721H16,2,1,3,2,0,0,3,2,2,0,1,0,0,0,0,5,0,0,0,0,5,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,6,0,0,0,0,0
7831,224475564,2,NaN,NaN,0.15,4,Noroeste,1,98,1988,5,NaN,5.0,contabilidad despacho,TRANSPORTES Y COMUNICACIONES,1,NaN,1.0,1,acon

Concatenamos los nulos, NS, NC en una sola categoría "NULO".

In [53]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
concat_null(df, vars_to_cat)

,history_id_crm,estrato_nuevo,estrato_nuevo_txt,ponderador_sgpff,pond_relativizado,...,p2_45_both,p2_45_contr98,p2_45_contr99,p2_45_imp98,p2_45_imp99
0,7521766,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,6,NULO,NULO,NULO,NULO
1,7544090,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,NULO,NULO,NULO,NULO,NULO
2,7543953,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,NULO,NULO,NULO,NULO,NULO
3,7536178,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,NULO,NULO,NULO,NULO,NULO
4,7521922,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,2,NULO,NULO,NULO,NULO
...,...,...,...,...,...,...,...,...,...,...,...
7829,224423829,3,NaN,NaN,0.13,...,NULO,NULO,NULO,NULO,NULO
7830,224637558,1,NaN,NaN,4.56,...,NULO,NULO,NULO,NULO,NULO
7831,224475564,2,NaN,NaN,0.15,...,NULO,NULO,NULO,NULO,NULO
7832,223989494,3,NaN,NaN,0.05,...,NULO,NULO,NULO,NULO,NULO


Convertimos todas las variables en catagóricas.

In [54]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
convert_object(df, vars_to_cat)

,history_id_crm,estrato_nuevo,estrato_nuevo_txt,ponderador_sgpff,pond_relativizado,...,p2_45_both,p2_45_contr98,p2_45_contr99,p2_45_imp98,p2_45_imp99
0,7521766,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,6,NULO,NULO,NULO,NULO
1,7544090,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,NULO,NULO,NULO,NULO,NULO
2,7543953,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,NULO,NULO,NULO,NULO,NULO
3,7536178,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,NULO,NULO,NULO,NULO,NULO
4,7521922,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,2,NULO,NULO,NULO,NULO
...,...,...,...,...,...,...,...,...,...,...,...
7829,224423829,3,NaN,NaN,0.13,...,NULO,NULO,NULO,NULO,NULO
7830,224637558,1,NaN,NaN,4.56,...,NULO,NULO,NULO,NULO,NULO
7831,224475564,2,NaN,NaN,0.15,...,NULO,NULO,NULO,NULO,NULO
7832,223989494,3,NaN,NaN,0.05,...,NULO,NULO,NULO,NULO,NULO


### Profiling de variables categóricas.

In [55]:
data_profiling_categ(df, vars_to_cat)

*********************************
Variable Categorica p1_2
*********************************


Info,p1_2
Num_Registros,7834
Num_de_categorias,7
Moda,5
Valores_faltantes,0
Top1,"[5, 3079]"
Top2,"[4, 1496]"
Top3,"[2, 1244]"


None
Valores de las categorias y sus proporciones


,Observaciones,proporción
Categoría,,
5,3079,39.3%
4,1496,19.1%
2,1244,15.9%
3,1155,14.7%
6,417,5.3%
1,360,4.6%
NULO,83,1.1%


None



*********************************
Variable Categorica p1_4
*********************************


...

None



*********************************
Variable Categorica p6_3
*********************************


Info,p6_3
Num_Registros,7834
Num_de_categorias,14
Moda,10
Valores_faltantes,0
Top1,"[10, 3017]"
Top2,"[11, 2974]"
Top3,"[9, 411]"


None
Valores de las categorias y sus proporciones


,Observaciones,proporción
Categoría,,
10,3017,38.5%
11,2974,38.0%
9,411,5.2%
12,400,5.1%
8,379,4.8%
...,...,...
3,31,0.4%
13,28,0.4%
4,7,0.1%


None





Porcentaje de valores nulos, NC, NS por pregunta:

In [260]:
null_bar_plot(df, vars_to_cat)